test cases:

1. get the expected output
2. can throw out errors (unfitted, format error, etc.)
3. produce correct (or nearly correct) amount of NAs

In [1]:
import pandas as pd
import numpy as np

In [2]:
%cd /Users/alex/PETsARD

/Users/alex/PETsARD


In [3]:
import PETsARD

In [4]:
loader = PETsARD.Loader(filepath = '[NHANES] B.csv',
                                header_exist = False,
                                header_names = ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
                               )
print(loader.data.head(1))

    gen   age   race       edu      mar        bmi  dep  pir   gh  mets  qm  \
0  Male  62.0  White  Graduate  Married  27.799999    0    0  0.0   0.0  Q2   

   dia  
0    1  


In [5]:
df = loader.data

In [6]:
m = PETsARD.Metadata()
m.build_metadata(df)

In [7]:
pm = PETsARD.Processor.Base.Processor(m)

No self-defined config passed.  Generate a config automatically.


In [8]:
from PETsARD.Processor.Missingist import *
from PETsARD.Processor.Outlierist import *
from PETsARD.Processor.Encoder import *
from PETsARD.Processor.Scaler import *

In [9]:
# df.loc[[1,2,3,4,5], 'gen'] = np.nan
# df.loc[[10,20,30,40,5], 'age'] = np.nan

In [14]:
# test global outlierist transformation

config_1 = {'missingist': {'gen': MissingistDrop(), 'age': MissingistSimple(5)},
            'outlierist': {'gen': None, 'age': OutlieristZScore()},
            'encoder': {'gen': EncoderLabel()},
            'scaler': {'age': ScalerLog()}}

In [9]:
config_2 = {'missingist': {'gen': MissingistDrop(), 'age': 'missingist_simple'},
            'outlierist': {'gen': None, 'age': 'outlierist_lof'},
            'encoder': {'gen': 'encoder_onehot', 'race': 'encoder_onehot'},
            'scaler': {'age': 'scaler_minmax'}}

In [10]:
pm.update_config(config_2)

In [8]:
pm.get_config()

{'missingist': {'gen': <PETsARD.Processor.Missingist.MissingistDrop at 0x1524c45b0>,
  'age': <PETsARD.Processor.Missingist.MissingistSimple at 0x1524c7bb0>,
  'race': <PETsARD.Processor.Missingist.MissingistDrop at 0x1524c7010>,
  'edu': <PETsARD.Processor.Missingist.MissingistDrop at 0x1524c6fb0>,
  'mar': <PETsARD.Processor.Missingist.MissingistDrop at 0x10464bd00>,
  'bmi': <PETsARD.Processor.Missingist.MissingistMean at 0x10444b100>,
  'dep': <PETsARD.Processor.Missingist.MissingistMean at 0x1524c6e90>,
  'pir': <PETsARD.Processor.Missingist.MissingistMean at 0x1524c6c80>,
  'gh': <PETsARD.Processor.Missingist.MissingistMean at 0x1524c67a0>,
  'mets': <PETsARD.Processor.Missingist.MissingistMean at 0x1524c75e0>,
  'qm': <PETsARD.Processor.Missingist.MissingistDrop at 0x1524c6aa0>,
  'dia': <PETsARD.Processor.Missingist.MissingistMean at 0x104494940>},
 'outlierist': {'gen': None,
  'age': <PETsARD.Processor.Outlierist.OutlieristLOF at 0x1524c5120>,
  'race': None,
  'edu': None,
 

In [9]:
pm.fit(df, sequence=['missingist', 'cube'])

ValueError: gh is constant. Please drop the data or change the data type to categorical.

In [17]:
df_transformed = pm.transform(df)

In [18]:
df_transformed

gen       age      race       edu       mar       bmi       dep  \
0     0.995762  0.700000  0.280727  0.434496  0.309646 -0.202041 -0.514504   
1     0.920824  0.550000  0.223206  0.618895  0.759233  0.235906 -0.514504   
2     0.649078  0.966667  0.015167  0.561078  0.316135 -0.056059 -0.514504   
3     0.065273  0.600000  0.140897  0.506134  0.857062  1.929305  1.943618   
4     0.300694  0.366667  0.355819  0.105264  0.746728 -1.296911  1.943618   
...        ...       ...       ...       ...       ...       ...       ...   
4162  0.794654  0.083333  0.761666  0.374957  0.561619 -1.194723 -0.514504   
4163  0.156598  0.933333  0.259884  0.536684  0.942874 -1.121732  1.943618   
4164  0.357402  1.000000  0.296827  0.120375  0.915073  0.265103 -0.514504   
4165  0.726288  0.250000  0.250947  0.775386  0.087906 -0.464810 -0.514504   
4166  0.284028  0.066667  0.125053  0.358368  0.608571 -1.136330 -0.514504   

           pir   gh  mets        qm       dia  
0    -0.517100  0.0   0.0  0.309048  1.949241  
1     1.933861  0.0   0.0  0.030044 -0.513020  
2    -0.517100  0.0   0.0  0.817670  1.949241  
3    -0.517100  0.0   0.0  0.890667 -0.513020  
4    -0.517100  0.0   0.0  0.573128 -0.513020  
...        ...  ...   ...       ...       ...  
4162 -0.517100  0.0   0.0  0.240880 -0.513020  
4163 -0.517100  0.0   0.0  0.061569 -0.513020  
4164 -0.517100  0.0   0.0  0.209028 -0.513020  
4165  1.933861  0.0   0.0  0.179578 -0.513020  
4166 -0.517100  0.0   0.0  0.911630 -0.513020  

[4167 rows x 12 columns]

In [19]:
df_inverse = pm.inverse_transform(df_transformed)

/Users/alex/miniforge3/envs/py310/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:441: RuntimeWarning: invalid value encountered in add
  bin_centers = (bin_edges[1:] + bin_edges[:-1]) * 0.5


In [20]:
df_inverse

,gen,age,race,edu,mar,bmi,dep,pir,gh,mets,qm,dia
0,Male,62.0,White,Graduate,Married,30.340000,0.1,0.1,0.1,NaN,Q2,0.9
1,Male,50.0,White,HighSchool,Divorced,30.340000,0.1,0.9,0.1,NaN,Q1,0.1
2,Male,74.0,White,HighSchool,Married,30.340000,0.1,0.1,0.9,NaN,Q3,0.9
3,Female,62.0,White,Graduate,Parther,40.900002,0.9,0.1,0.1,NaN,Q3,0.1
4,Female,38.0,Black,College,Divorced,19.780001,0.9,0.1,0.1,NaN,Q4,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...
4185,Male,26.0,Other,Graduate,Never,19.780001,0.1,0.1,0.1,NaN,Q1,0.1
4186,Female,74.0,White,HighSchool,Widowed,19.780001,0.9,0.1,0.1,NaN,Q1,0.1
4187,Female,74.0,White,College,Widowed,30.340000,0.1,0.1,0.1,NaN,Q1,0.1
4188,Male,38.0,White,9th,Married,30.340000,0.1,0.9,0.1,NaN,Q1,0.1


In [23]:
try:
    print(f'Inverse successful:\n{(df_inverse == df).all()}')
    print(f'Numeric close (age): {np.isclose(df_inverse.age, df.age).all()}')
    print(f'Numeric close (bmi): {np.isclose(df_inverse.bmi, df.bmi).all()}')
except:
    print('Not applicable.')

Not applicable.


In [19]:
loader2 = PETsARD.Loader(filepath = '[Adt Income] adult.csv'
                               ,na_values = {k : '?' for k in ['workclass'
                                                              ,'occupation'
                                                              ,'native-country'
                                                              ]
                                            }
                               )
print(loader2.data.head(1))

   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [20]:
df2 = loader2.data

In [21]:
metadata2 = {'metadata_col': {
                'age': {'type': 'numerical', 'na_percentage': 0.0},
                'workclass': {'type': 'categorical', 'na_percentage': 0.057307},
                'fnlwgt': {'type': 'numerical', 'na_percentage': 0.0},
                'education': {'type': 'categorical', 'na_percentage': 0.0},
                'educational-num': {'type': 'numerical', 'na_percentage': 0.0},
                'marital-status': {'type': 'categorical', 'na_percentage': 0.0},
                'occupation': {'type': 'categorical', 'na_percentage': 0.057512},
                'relationship': {'type': 'categorical', 'na_percentage': 0.0},
                'race': {'type': 'categorical', 'na_percentage': 0.0},
                'gender': {'type': 'categorical', 'na_percentage': 0.0},
                'capital-gain': {'type': 'numerical', 'na_percentage': 0.0},
                'capital-loss': {'type': 'numerical', 'na_percentage': 0.0},
                'hours-per-week': {'type': 'numerical', 'na_percentage': 0.0},
                'native-country': {'type': 'categorical', 'na_percentage': 0.017546},
                'income': {'type': 'categorical', 'na_percentage': 0.0}
                },
             'metadata_global':{
                 'row_num': 48842,
                 'col_num': 15,
                 'na_percentage': 0.07411653904426518
                }
            }

In [22]:
m2 = PETsARD.Metadata()
m2.build_metadata(df2)

In [23]:
pm2 = PETsARD.Processor.Base.Processor(m2)

No self-defined config passed.  Generate a config automatically.


In [24]:
pm2.get_config()

{'missingist': {'age': <PETsARD.Processor.Missingist.MissingistMean at 0x1525636d0>,
  'workclass': <PETsARD.Processor.Missingist.MissingistDrop at 0x152561600>,
  'fnlwgt': <PETsARD.Processor.Missingist.MissingistMean at 0x152561480>,
  'education': <PETsARD.Processor.Missingist.MissingistDrop at 0x1525614e0>,
  'educational-num': <PETsARD.Processor.Missingist.MissingistMean at 0x1525631c0>,
  'marital-status': <PETsARD.Processor.Missingist.MissingistDrop at 0x1525629b0>,
  'occupation': <PETsARD.Processor.Missingist.MissingistDrop at 0x152562260>,
  'relationship': <PETsARD.Processor.Missingist.MissingistDrop at 0x1525637c0>,
  'race': <PETsARD.Processor.Missingist.MissingistDrop at 0x1525626e0>,
  'gender': <PETsARD.Processor.Missingist.MissingistDrop at 0x152561cc0>,
  'capital-gain': <PETsARD.Processor.Missingist.MissingistMean at 0x152563490>,
  'capital-loss': <PETsARD.Processor.Missingist.MissingistMean at 0x152563ca0>,
  'hours-per-week': <PETsARD.Processor.Missingist.Missingi

In [25]:
pm2.fit(df2)

In [26]:
pm2.get_config()

{'missingist': {'age': <PETsARD.Processor.Missingist.MissingistMean at 0x1525636d0>,
  'workclass': <PETsARD.Processor.Missingist.MissingistDrop at 0x152561600>,
  'fnlwgt': <PETsARD.Processor.Missingist.MissingistMean at 0x152561480>,
  'education': <PETsARD.Processor.Missingist.MissingistDrop at 0x1525614e0>,
  'educational-num': <PETsARD.Processor.Missingist.MissingistMean at 0x1525631c0>,
  'marital-status': <PETsARD.Processor.Missingist.MissingistDrop at 0x1525629b0>,
  'occupation': <PETsARD.Processor.Missingist.MissingistDrop at 0x152562260>,
  'relationship': <PETsARD.Processor.Missingist.MissingistDrop at 0x1525637c0>,
  'race': <PETsARD.Processor.Missingist.MissingistDrop at 0x1525626e0>,
  'gender': <PETsARD.Processor.Missingist.MissingistDrop at 0x152561cc0>,
  'capital-gain': <PETsARD.Processor.Missingist.MissingistMean at 0x152563490>,
  'capital-loss': <PETsARD.Processor.Missingist.MissingistMean at 0x152563ca0>,
  'hours-per-week': <PETsARD.Processor.Missingist.Missingi

In [27]:
df_transformed2 = pm2.transform(df2)

In [28]:
df_transformed2

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,-0.995129,0.644535,0.351675,0.833907,-1.197259,0.665872,0.755129,0.682417,0.892638,0.324092,-0.144804,-0.217127,-0.034087,0.225421,0.702293
1,-0.046942,0.228809,-0.945524,0.085026,-0.419335,0.328718,0.939339,0.309968,0.131591,0.408057,-0.144804,-0.217127,0.772930,0.196446,0.268079
2,-0.776316,0.823952,1.394723,0.861193,0.747550,0.410292,0.987816,0.342936,0.659045,0.414243,-0.144804,-0.217127,-0.034087,0.883711,0.901505
3,-1.068066,0.529774,1.704525,0.455411,-0.030373,0.613680,0.746867,0.839017,0.051014,0.916966,-0.144804,-0.217127,-0.034087,0.032184,0.251911
4,-0.192816,0.997546,0.215911,0.585863,1.136512,0.430817,0.444799,0.317257,0.378977,0.250399,-0.144804,-0.217127,-0.034087,0.446325,0.473624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26928,1.047121,0.355549,1.251867,0.763439,1.525474,0.057031,0.388025,0.039223,0.649175,0.341987,-0.144804,-0.217127,-0.034087,0.506468,0.849765
26929,-1.213941,0.306103,1.140952,0.451630,-0.030373,0.707860,0.994031,0.441084,0.036823,0.560695,-0.144804,-0.217127,-0.034087,0.782466,0.750509
26930,-0.849254,0.004884,0.640492,0.865394,0.747550,0.244096,0.945848,0.939025,0.244672,0.682935,-0.144804,-0.217127,-0.195490,0.562829,0.126520
26931,0.098933,0.140005,-0.334178,0.128162,-0.419335,0.209549,0.785479,0.358074,0.371285,0.289620,-0.144804,-0.217127,-0.034087,0.811582,0.840769


In [29]:
df_inverse2 = pm2.inverse_transform(df_transformed2)

In [30]:
df_inverse2

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,24.0,Private,369667.0,Some-college,10.0,Never-married,Other-service,Unmarried,White,Female,0.0,0.0,40.0,United-States,<=50K
4,36.0,NaN,212465.0,Bachelors,13.0,Married-civ-spouse,NaN,Husband,White,Male,0.0,0.0,40.0,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26928,53.0,Private,321865.0,Masters,14.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
26929,22.0,Private,310152.0,Some-college,10.0,Never-married,NaN,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
26930,27.0,Private,257302.0,Assoc-acdm,12.0,Married-civ-spouse,Tech-support,Wife,White,Female,0.0,0.0,38.0,United-States,<=50K
26931,40.0,Private,154374.0,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0.0,0.0,40.0,United-States,>50K


In [31]:
df2

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [32]:
df_inverse2.isna().mean(axis=0)

age                0.000000
workclass          0.057290
fnlwgt             0.000000
education          0.000000
educational-num    0.000000
marital-status     0.000000
occupation         0.057476
relationship       0.000000
race               0.000000
gender             0.000000
capital-gain       0.000000
capital-loss       0.000000
hours-per-week     0.000000
native-country     0.017525
income             0.000000
dtype: float64

In [38]:
df_inverse2.isna().any(axis=1).mean()

0.07110236512828129

In [39]:
pm.get_changes()

,processor,col,current,default
0,missingist,age,MissingistSimple,MissingistMean
1,outlierist,gen,OutlieristLOF,NoneType
2,outlierist,age,OutlieristLOF,OutlieristIQR
3,outlierist,race,OutlieristLOF,NoneType
4,outlierist,edu,OutlieristLOF,NoneType
5,outlierist,mar,OutlieristLOF,NoneType
6,outlierist,bmi,OutlieristLOF,OutlieristIQR
7,outlierist,dep,OutlieristLOF,OutlieristIQR
8,outlierist,pir,OutlieristLOF,OutlieristIQR
9,outlierist,gh,OutlieristLOF,OutlieristIQR


In [40]:
pm._default_processor['outlierist']['categorical']

<function PETsARD.Processor.Base.Processor.__init__.<locals>.<lambda>()>

In [41]:
pm._default_processor['outlierist']['numerical']

PETsARD.Processor.Outlierist.OutlieristIQR

In [42]:
from sdv.metadata import SingleTableMetadata

In [40]:
sdvmd = SingleTableMetadata()

In [41]:
sdvmd.detect_from_dataframe(df)

In [42]:
t = sdvmd.to_dict()

In [43]:
sdvmd2 = SingleTableMetadata()

In [44]:
mm = m.to_sdv()

Metadata Example

In [7]:
metadata = {'col': {
                'gen': {'type': 'categorical', 'na_percentage': 0.0},
                'age': {'type': 'numerical', 'na_percentage': 0.0},
                'race': {'type': 'categorical', 'na_percentage': 0.0},
                'edu': {'type': 'categorical', 'na_percentage': 0.0},
                'mar': {'type': 'categorical', 'na_percentage': 0.0},
                'bmi': {'type': 'numerical', 'na_percentage': 0.0},
                'dep': {'type': 'numerical', 'na_percentage': 0.0},
                'pir': {'type': 'numerical', 'na_percentage': 0.0},
                'gh': {'type': 'numerical', 'na_percentage': 0.0},
                'mets': {'type': 'numerical', 'na_percentage': 0.0},
                'qm': {'type': 'categorical', 'na_percentage': 0.0},
                'dia': {'type': 'numerical', 'na_percentage': 0.0}
                },
             'global':{
                 'row_num': 4190,
                 'col_num': 12,
                 'na_percentage': 0.0
                }
            }